In [ ]:
import random
from datetime import datetime, timedelta

class DriverTypeA:
    def __init__(self, driver_id):
        self.driver_id = driver_id
        self.type = 'A'
        self.working_hours = 8  # данный тип водителей работают 8 часов в день
        self.lunch_duration = timedelta(hours=1)  # обед занимает один час
        self.min_work_before_lunch = timedelta(hours=4)  # минимальное время работы перед обедом

    def can_work(self, day):
        return day.weekday() < 5  # водители типа А работают только в будние дни

class DriverTypeB:
    def __init__(self, driver_id):
        self.driver_id = driver_id
        self.type = 'B'
        self.shift_pattern = [1, 0, 0]  # график работы 1/2
        self.break_duration = timedelta(minutes=20)  # перерыв длится 20 минут
        self.lunch_duration = timedelta(minutes=40)  # обед - 40 минут
        self.current_day = 0  # текущий
        self.last_work_day = -3  # последний рабочий день (чтобы была возможность работать в понедельник, так как мы проверяем, что водители типа В отдыхали до этого два дня, чтобы начать работу)

    def can_work(self, day): # проверяем, прошло ли два дня с последнего рабочего дня водителя типа В
        shift_day = (day.weekday() + self.current_day) % 3
        if self.shift_pattern[shift_day] == 1 and (day.weekday() - self.last_work_day >= 2 or self.last_work_day == -3):
            return True
        return False

    def next_day(self):
        self.current_day += 1

    def set_last_work_day(self, day):
        self.last_work_day = day.weekday()

class Bus:
    def __init__(self, bus_id):
        self.bus_id = bus_id
        self.route_duration = timedelta(hours=1, minutes=10)  # полный маршрут занимает 1 час 10 минут

class Schedule:
    def __init__(self):
        self.drivers = []
        self.buses = []
        self.schedule = {}
        self.rush_hour_drivers = []  # водители для часа-пик
        self.rush_hour_buses = []  # автобусы для часа-пик

    def add_driver(self, driver):
        self.drivers.append(driver)

    def add_bus(self, bus):
        self.buses.append(bus)

    def add_rush_hour_driver(self, driver):
        self.rush_hour_drivers.append(driver)

    def add_rush_hour_bus(self, bus):
        self.rush_hour_buses.append(bus)

    def buy_more_buses_and_drivers(self, num_buses, num_drivers_a, num_drivers_b):
        for i in range(num_buses):
            self.add_bus(Bus(len(self.buses) + 1))  # добавление новых автобусов
        for i in range(num_drivers_a):
            self.add_rush_hour_driver(DriverTypeA(len(self.drivers) + len(self.rush_hour_drivers) + 1))  # добавление новых водителей типа А
        for i in range(num_drivers_b):
            self.add_rush_hour_driver(DriverTypeB(len(self.drivers) + len(self.rush_hour_drivers) + 1))  # добавление новых водителей типа В

    def generate_schedule(self):
        start_time = datetime.strptime('06:00', '%H:%M')  # начало рабочего дня в 6 утра
        end_time = datetime.strptime('03:00', '%H:%M') + timedelta(days=1)  # конец рабочего дня в 3 часа ночи

        # распределение водителей типа В в будние дни
        type_b_drivers = [driver for driver in self.drivers if isinstance(driver, DriverTypeB)]
        type_b_schedule = {0: [], 1: [], 2: [], 3: [], 4: []}
        for i, driver in enumerate(type_b_drivers):
            day = i % 5
            type_b_schedule[day].append(driver)

        for day in range(7):
            self.schedule[day] = {}
            current_day = start_time + timedelta(days=day)
            current_time = start_time

            available_drivers = [driver for driver in self.drivers if driver.can_work(current_day)]
            available_drivers.sort(key=lambda x: x.driver_id)  # сортировка водителей по ID

            # водители типа В могут вернуться к работе спустя 2 выходных дня
            for driver in self.drivers:
                if isinstance(driver, DriverTypeB):
                    driver.next_day()

            # добавление одного водителя типа В на каждый день
            if day < 5:
                available_drivers.extend(type_b_schedule[day][:1])  # только один водитель типа В из оригинальной восьмёрки водителей
            else:
                available_drivers.extend(type_b_drivers[:2])

            # определить нужное количество автобусов
            num_buses_needed = self.calculate_buses_needed(current_day, current_time)

            # выдать водителям необходимое количество автобусов
            for i in range(num_buses_needed):
                if i >= len(available_drivers):
                    break
                driver = available_drivers[i]
                driver_schedule = []
                if i == 0:
                    work_start_time = current_time
                else:
                    work_start_time = self.get_staggered_start_time(i, work_start_time)

                lunch_taken = False

                if isinstance(driver, DriverTypeA):
                    work_end_time = work_start_time + timedelta(hours=driver.working_hours)
                elif isinstance(driver, DriverTypeB):
                    work_end_time = end_time  # водители типа В работают весь день

                current_time = work_start_time

                while current_time < work_end_time and current_time < end_time:
                    if isinstance(driver, DriverTypeA):
                        if not lunch_taken and (current_time - work_start_time) >= driver.min_work_before_lunch:
                            driver_schedule.append((current_time, current_time + driver.lunch_duration, 'Обед'))
                            current_time += driver.lunch_duration
                            lunch_taken = True
                        else:
                            driver_schedule.append((current_time, current_time + Bus(1).route_duration, 'Маршрут'))
                            current_time += Bus(1).route_duration
                    elif isinstance(driver, DriverTypeB):
                        if not lunch_taken and (current_time - work_start_time) >= timedelta(hours=2):
                            driver_schedule.append((current_time, current_time + driver.lunch_duration, 'Обед'))
                            current_time += driver.lunch_duration
                            lunch_taken = True
                        else:
                            driver_schedule.append((current_time, current_time + Bus(1).route_duration, 'Маршрут'))
                            current_time += Bus(1).route_duration
                            if (current_time - work_start_time) // timedelta(hours=1) % 2 == 0 and not lunch_taken:
                                driver_schedule.append((current_time, current_time + driver.break_duration, 'Перерыв'))
                                current_time += driver.break_duration
                            elif lunch_taken:
                                driver_schedule.append((current_time, current_time + driver.break_duration, 'Перерыв'))
                                current_time += driver.break_duration
                                driver_schedule.append((current_time, current_time + driver.break_duration, 'Перерыв'))
                                current_time += driver.break_duration

                self.schedule[day][driver.driver_id] = driver_schedule
                if isinstance(driver, DriverTypeB):
                    driver.set_last_work_day(current_day)

            # график водителей в час-пик
            self.schedule_rush_hour_drivers(day, current_day, start_time, end_time)

    def schedule_rush_hour_drivers(self, day, current_day, start_time, end_time):
        morning_rush_start = datetime.strptime('07:00', '%H:%M')  # утренний час-пик в 7 утра
        evening_rush_start = datetime.strptime('17:00', '%H:%M')  # вечерний час-пик в 5 вечера

        # водитель типа А под утренний час-пик в будние дни
        if day < 5:
            driver = self.rush_hour_drivers[0]
            driver_schedule = []
            work_start_time = morning_rush_start
            work_end_time = work_start_time + timedelta(hours=8)
            current_time = work_start_time
            lunch_taken = False

            while current_time < work_end_time:
                if not lunch_taken and (current_time - work_start_time) >= driver.min_work_before_lunch:
                    driver_schedule.append((current_time, current_time + driver.lunch_duration, 'Обед'))
                    current_time += driver.lunch_duration
                    lunch_taken = True
                else:
                    driver_schedule.append((current_time, current_time + Bus(1).route_duration, 'Маршрут'))
                    current_time += Bus(1).route_duration

            self.schedule[day][driver.driver_id] = driver_schedule

        # водитель типа А под вечерний час-пик в будние дни
        if day < 5:
            driver = self.rush_hour_drivers[1]
            driver_schedule = []
            work_start_time = evening_rush_start
            work_end_time = work_start_time + timedelta(hours=8)
            current_time = work_start_time
            lunch_taken = False

            while current_time < work_end_time:
                if not lunch_taken and (current_time - work_start_time) >= driver.min_work_before_lunch:
                    driver_schedule.append((current_time, current_time + driver.lunch_duration, 'Обед'))
                    current_time += driver.lunch_duration
                    lunch_taken = True
                else:
                    driver_schedule.append((current_time, current_time + Bus(1).route_duration, 'Маршрут'))
                    current_time += Bus(1).route_duration

            self.schedule[day][driver.driver_id] = driver_schedule

        # дополнительные водители типа В в будние дни
        if day < 5:
            available_b_drivers = [driver for driver in self.rush_hour_drivers if isinstance(driver, DriverTypeB) and driver.can_work(current_day)]
            if available_b_drivers:
                driver = available_b_drivers[0]
                driver_schedule = []
                work_start_time = evening_rush_start
                work_end_time = work_start_time + timedelta(hours=8)
                current_time = work_start_time
                lunch_taken = False

                while current_time < work_end_time:
                    if not lunch_taken and (current_time - work_start_time) >= timedelta(hours=2):
                        driver_schedule.append((current_time, current_time + driver.lunch_duration, 'Обед'))
                        current_time += driver.lunch_duration
                        lunch_taken = True
                    else:
                        driver_schedule.append((current_time, current_time + Bus(1).route_duration, 'Маршрут'))
                        current_time += Bus(1).route_duration
                        if (current_time - work_start_time) // timedelta(hours=1) % 2 == 0 and not lunch_taken:
                            driver_schedule.append((current_time, current_time + driver.break_duration, 'Перерыв'))
                            current_time += driver.break_duration
                        elif lunch_taken:
                            driver_schedule.append((current_time, current_time + driver.break_duration, 'Перерыв'))
                            current_time += driver.break_duration
                            driver_schedule.append((current_time, current_time + driver.break_duration, 'Перерыв'))
                            current_time += driver.break_duration

                self.schedule[day][driver.driver_id] = driver_schedule
                driver.set_last_work_day(current_day)

        # дополнительные водители типа В в выходные дни
        if day >= 5:
            additional_b_drivers = [driver for driver in self.rush_hour_drivers if isinstance(driver, DriverTypeB) and driver.can_work(current_day)]
            for i, driver in enumerate(additional_b_drivers[:3]):  # можно использовать до трёх дополнительных водителей
                driver_schedule = []
                work_start_time = start_time
                work_end_time = end_time
                current_time = work_start_time
                lunch_taken = False

                while current_time < work_end_time:
                    if not lunch_taken and (current_time - work_start_time) >= timedelta(hours=2):
                        driver_schedule.append((current_time, current_time + driver.lunch_duration, 'Обед'))
                        current_time += driver.lunch_duration
                        lunch_taken = True
                    else:
                        driver_schedule.append((current_time, current_time + Bus(1).route_duration, 'Маршрут'))
                        current_time += Bus(1).route_duration
                        if (current_time - work_start_time) // timedelta(hours=1) % 2 == 0 and not lunch_taken:
                            driver_schedule.append((current_time, current_time + driver.break_duration, 'Перерыв'))
                            current_time += driver.break_duration
                        elif lunch_taken:
                            driver_schedule.append((current_time, current_time + driver.break_duration, 'Перерыв'))
                            current_time += driver.break_duration
                            driver_schedule.append((current_time, current_time + driver.break_duration, 'Перерыв'))
                            current_time += driver.break_duration

                self.schedule[day][driver.driver_id] = driver_schedule
                driver.set_last_work_day(current_day)

    def get_staggered_start_time(self, driver_index, previous_start_time):
        # рандомный запуск, начиная со второго водителя, с периодом от 30 до 180 минут
        if driver_index == 0:
            return previous_start_time
        else:
            return previous_start_time + timedelta(minutes=random.randint(30, 180))

    def calculate_buses_needed(self, day, current_time):
        if day.weekday() < 5:
            if 7 <= current_time.hour < 9 or 17 <= current_time.hour < 19:
                return 10  # использование до 10 водителей в час-пик
            else:
                return 8  # 8 водителей в остальные часы
        else:
            return 8  # также используем до 8 водителей в выходные

    def print_schedule(self):
        days_of_week = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
        for day, driver_schedules in self.schedule.items():
            print(f"{days_of_week[day]}:")
            for driver_id in sorted(driver_schedules.keys()):
                driver = next((driver for driver in self.drivers + self.rush_hour_drivers if driver.driver_id == driver_id), None)
                if driver:
                    driver_type = driver.type
                    print(f"Водитель {driver_id} ({driver_type}):")
                    for event in driver_schedules[driver_id]:
                        start_time, end_time, event_type = event
                        print(f"{start_time.strftime('%H:%M')} - {end_time.strftime('%H:%M')} ({event_type})")
                    print()

class GeneticAlgorithm:
    def __init__(self, population_size, mutation_rate, generations):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.population = []

    def initialize_population(self):
        for _ in range(self.population_size):
            schedule = Schedule()
            for i in range(1, 5):
                schedule.add_driver(DriverTypeA(i))
            for i in range(5, 9):
                schedule.add_driver(DriverTypeB(i))
            for i in range(1, 9):
                schedule.add_bus(Bus(i))
            schedule.buy_more_buses_and_drivers(3, 2, 5)
            schedule.generate_schedule()
            self.population.append(schedule)

    def fitness(self, schedule):
        # фитнес функция минимизирует время начала первой поездки второго водителя
        monday_schedule = schedule.schedule.get(0, {})
        if len(monday_schedule) < 2:
            return float('inf')  # если водителей меньше, чем два, то фитнес бесконечно огромный
        second_driver_start_time = monday_schedule[sorted(monday_schedule.keys())[1]][0][0]
        target_time = datetime.strptime('06:30', '%H:%M')
        time_difference = abs((second_driver_start_time - target_time).total_seconds())
        return time_difference

    def select_parents(self):
        # выбор родителей
        selected = random.sample(self.population, k=2)
        selected.sort(key=self.fitness)
        return selected[0]

    def crossover(self, parent1, parent2):
        child = Schedule()
        for i in range(1, 5):
            child.add_driver(DriverTypeA(i))
        for i in range(5, 9):
            child.add_driver(DriverTypeB(i))
        for i in range(1, 9):
            child.add_bus(Bus(i))
        child.buy_more_buses_and_drivers(3, 2, 5)

        # кроссовер, т.е. смена генов родителей и получение потомства
        for day in range(7):
            if random.random() < 0.5:
                child.schedule[day] = parent1.schedule[day]
            else:
                child.schedule[day] = parent2.schedule[day]

        return child

    def mutate(self, schedule):
        # рандомная мутация, изменяющая время выезда водителя
        if random.random() < self.mutation_rate:
            monday_schedule = schedule.schedule.get(0, {})
            if monday_schedule:
                first_driver_id = sorted(monday_schedule.keys())[0]
                new_start_time = datetime.strptime('06:00', '%H:%M') + timedelta(minutes=random.randint(0, 180))
                monday_schedule[first_driver_id][0] = (new_start_time, new_start_time + Bus(1).route_duration, 'Маршрут')

    def evolve(self):
        # создание нового поколения
        self.initialize_population()
        for generation in range(self.generations):
            new_population = []
            for _ in range(self.population_size):
                parent1 = self.select_parents()
                parent2 = self.select_parents()
                child = self.crossover(parent1, parent2)
                self.mutate(child)
                new_population.append(child)
            self.population = new_population
            best_schedule = min(self.population, key=self.fitness)
            print(f"Поколение {generation + 1}, Лучший фитнес: {self.fitness(best_schedule)}")

        best_schedule = min(self.population, key=self.fitness)
        best_schedule.print_schedule()

# запуск генетического алгоритма и вывод результата
ga = GeneticAlgorithm(population_size=10, mutation_rate=0.1, generations=50)
ga.evolve()
